# Filesystem Stuff

## Paths

Julia provides a lot of built-ins for working with paths in a
cross-platformy way.

In [1]:
currdir = pwd()
@show basename(currdir)
@show dirname(currdir)
readme = joinpath(currdir, "README.rst")

basename(currdir) = "administrative-scripting-with-julia"
dirname(currdir) = "/home/ninjaaron/doc"


"/home/ninjaaron/doc/administrative-scripting-with-julia/README.rst"

`joinpath` can join an arbitrary number of path elements. I found it
very strange that there was no `splitpath` method to return an array
of all path elements. There has only been a `splitdir` function, which
returns a tuple.

In [2]:
splitdir(currdir)

("/home/ninjaaron/doc", "administrative-scripting-with-julia")

However, I'm happy to say that a `splitpath` method is included in the
1.1 release of Julia, for which a release candidate has just been
released (on 2019-1-1), so you should be able to do

```julia
julia> splitpath(currdir)
["/", "home", "ninjaaron", "doc", "administrative-scripting-with-julia"]
```

... or something like that.

In [3]:
@show splitext("README.rst")
@show isdir(readme)
@show isfile(readme)
st = stat(readme)

splitext("README.rst") = ("README", ".rst")
isdir(readme) = false
isfile(readme) = true


StatStruct(mode=0o100644, size=7528)

`StatStruct` instances have a lot more attributes than this, of
course. They have [all these
attributes](https://docs.julialang.org/en/v1/base/file/#Base.stat) as
well. A couple of these attributes, like `mtime` and `ctime` are in
Unix time, so it might be good mention that you can convert them to a
human readable representation with the Dates module, which is in the
standard library. It will be covered more in a later section. (Note
that this pretty-printed date is just the way it prints. It is a data
structure.)

In [4]:
import Dates
Dates.unix2datetime(st.mtime)

2019-01-02T12:58:42.201

There are many other methods available in Base which have names you should already recognize, which I won't demonstrate now. Names include: `cd`, `rm`, `mkdir`, `mkpath` (like `mkdir -p` in the shell), `symlink`, `chown`, `chmod` (careful to make sure youre mode argument is in octal, `0o644` or whatever), `cp`, `mv`, `touch`, as well as a lot of tests like `isfile`, `isdir`, `islink`, `isfifo`, etc. You know what they do, and you can [read the docs] if you need more. The one thing that's missing is `ls`. That's called `readdir`.

In [5]:
readdir()

7-element Array{String,1}:
 ".git"              
 ".gitignore"        
 ".ipynb_checkpoints"
 "CLI.ipynb"         
 "README.rst"        
 "files.ipynb"       
 "filesystem.ipynb"  

There's also a
[`walkdir`](https://docs.julialang.org/en/v1/base/file/#Base.Filesystem.walkdir)
which recursively walks the directory and returns tuples of
`(rootpath, dirs, files)` which is rather handy.

There are a few things Julia still lacks in the filesystem
department. It doesn't support any kind of file globbing, but that's
easy enough to handle with regex or plain substring matching.

In [6]:
[path for path in readdir() if occursin("ipynb", path)]

4-element Array{String,1}:
 ".ipynb_checkpoints"
 "CLI.ipynb"         
 "files.ipynb"       
 "filesystem.ipynb"  

In [7]:
# or
filter!(p -> !startswith(p, "."), readdir())

4-element Array{String,1}:
 "CLI.ipynb"       
 "README.rst"      
 "files.ipynb"     
 "filesystem.ipynb"

It also weirdly lacks a function for making hard links. Bah. I guess
that's what the [C interface](https://docs.julialang.org/en/v1/manual/calling-c-and-fortran-code/)
is for. (I'm both thumping my chest and groaning inside as I say that,
but at least it is crazy easy to call C from Julia and is as efficient
as native calls)

In [8]:
function hardlink(oldpath, newpath)
    # calling:  int link(char *oldpath, char *newpath)
    ret_code = ccall(:link, Int32, (Cstring, Cstring), oldpath, newpath)
    ret_code == 0 ? newpath : error("couldn't link link: $oldpath -> $newpath")
end

hardlink("README.rst", "foo.txt")
@show stat("foo.txt").nlink
rm("foo.txt")

(stat("foo.txt")).nlink = 2


Course, using `ccall` sort of depends on, you know, knowing enough C
to read and understand C function declarations for simple things, and
it involves pointers and memory allocation crap if you want to do
something more serious. It's C. What did you expect?

Julia also lacks Python's easy, built-in support for compression and
archive formats, though third-party packages do exist for GZip and Zip
archives. Maybe I should work on an archiving library. Hm.

Anyhow, there's more than one way to skin that cat. One distinctive
feature of Julia is that is very clear after you use it a little, but
it's hard to point to any one thing, is that it wants to make it easy
to bootstrap whatever functionality you need into the language. The
`ccall` API is part of that. It is used liberally in the
implementation of OS interfaces, as well as some of the mathematical
libraries (`ccall` also works on Fortran). Though they aren't shipped
with Julia, the community also maintain PyCall.jl and RCall.jl, which
allow "native" calls into those runtimes for wrapping their
libraries. Macros are different example of the same thing. Language
missing a feature? Alter the semantics with a macro. Yet another
example of this "bootstrap-ability" of Julia is the ease with which it
allows the programmer to orchestrate the use of external processes.

To take the example of the above `hardlink` function, If programming
in C ain't your bag, Julia has really great support for running
external processes, so it is also possible (but rather slower) to
simply do:

In [ ]:
hardlink(oldpath, newpath) = run(`link $oldpath $newpath`)